# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [59]:
# %load ../drawingTools
import ROOT
import math
import uuid
import pandas as pd

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 12)
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext

def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend

def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas

def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         min_ratio_y=None,
         max_ratio_y=None,
         min_x=None,
         max_x=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False):
    global colors
    global stuff
    global p_idx


    # 0 - determine kind and # of histos
    n_histos = len(histograms)
    if n_histos == 0:
        print '[draw]**ERROR: empy histo list'
        return

    histo_class = histograms[0].ClassName()

    # 1 - check argument consisntency
    if 'TH2' not in histo_class and do_profile:
        print '[draw]**ERROR: do_profile set for histo of class: {}'.format(histo_class)
        return

    # TH1 histograms are overlayed
    # TH2 histograms are drawn side by side in the same canvas
    overlay = True
    if n_histos > 1:
        if 'TH2' in histo_class or 'TH3' in histo_class:
            overlay = False

    # 2 - determine canvas properties
    xdiv = 0
    ydiv = 0
    c_width = 800
    c_height = 600
    # do_ratio = False

    if do_ratio:
        c_height = 800

    if not overlay:
        xdiv = 2
        c_width = 1000.
        ydiv = math.ceil(float(n_histos)/2)
        c_height = 500*ydiv
    canvas = newCanvas(name=None,
                       title=None,
                       height=int(c_height),
                       width=int(c_width),
                       xdiv=int(xdiv),
                       ydiv=int(ydiv))
    if overlay:
        canvas.SetRightMargin(0.30)

    canvas.cd()
    leg = getLegend()

    drawn_histos = []
    # drawn_histos = histograms
    for hidx, hist in enumerate(histograms):
        opt = options
        if 'TGraph' not in histo_class:
            hist.SetStats(do_stats)
        else:
            opt = 'P'+options
        if overlay:
            hist.SetLineColor(colors[hidx])
            if 'TGraph' in histo_class:
                hist.SetMarkerColor(colors[hidx])
            if hidx:
                opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'
        else:
            canvas.cd(hidx+1)
        # print hidx, opt
        d_hist = hist
        if norm:
            d_hist = hist.DrawNormalized(opt, 1.)
        else:
            hist.Draw(opt)

        if do_profile:
            profname = d_hist.GetName()+'_prof_'+str(p_idx)
            p_idx += 1
            firstbin = 1
            lastbin = -1
            prof = d_hist.ProfileX(profname, firstbin, lastbin, 's')
            prof.SetMarkerColor(2)
            prof.SetLineColor(2)
            prof.Draw('same')

        if overlay:
            if 'TGraph' not in histo_class:
                leg.AddEntry(hist, labels[hidx], 'l')
            else:
                leg.AddEntry(hist, labels[hidx], 'P')
        else:
            if text:
                newtext = '{}: {}'.format(labels[hidx], text)
                rtext = getText(newtext, 0.15, 0.85)
                rtext.Draw('same')
        drawn_histos.append(d_hist)

    if do_ratio:
        h_ratio = ROOT.TRatioPlot(drawn_histos[0], drawn_histos[1])
        stuff.append(h_ratio)
        h_ratio.Draw()

        pad = canvas.cd(1)
        h_ratio.GetUpperPad().SetBottomMargin(0)
        h_ratio.GetUpperPad().SetRightMargin(0.3)
        if logy:
            h_ratio.GetUpperPad().SetLogy()

        if min_x is not None and max_x is not None:
            h_ratio.GetUpperRefXaxis().SetRangeUser(min_x, max_x)
            h_ratio.GetLowerRefXaxis().SetRangeUser(min_x, max_x)

        pad.Update()
        pad = canvas.cd(2)
        h_ratio.GetLowerPad().SetTopMargin(0)
        h_ratio.GetLowerPad().SetRightMargin(0.3)
        

        if min_ratio_y is not None and max_ratio_y is not None:
            h_ratio.GetLowerRefGraph().SetMinimum(min_ratio_y)
            h_ratio.GetLowerRefGraph().SetMaximum(max_ratio_y)
        pad.Update()
        # hratio.GetLowerRefGraph().GetYaxis().SetRangeUser(-6, 6)

    canvas.Update()

    # we now set the axis properties
    max_value = max_y
    min_value = min_y

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos])*1.2
    # print min_value, max_value

    if not do_ratio and min_x is not None and max_x is not None:
        hist.GetXaxis().SetRangeUser(min_x, max_x)
    
    for hist in drawn_histos:
        hist.GetYaxis().SetRangeUser(min_value, max_value)
        if y_axis_label:
            hist.GetYaxis().SetTitle(y_axis_label)
        if x_axis_label:
            hist.GetXaxis().SetTitle(x_axis_label)

    canvas.Draw()

    # we draw additional stuff if needed
    if overlay:
        canvas.cd()
        leg.Draw()
        if text:
            rtext = getText(text, 0.15, 0.85)
            rtext.Draw("same")

    pad_range = range(0, 1)
    if not overlay:
        pad_range = range(1, len(histograms)+1)

    for pad_id in pad_range:
        canvas.cd(pad_id)
        if v_lines:
            for v_line_x in v_lines:
                aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if h_lines:
            for h_line_y in h_lines:
                aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if logy:
            if not do_ratio:
                ROOT.gPad.SetLogy()

        ROOT.gPad.Update()
    canvas.Update()
    canvas.Draw()

    return canvas



files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = type


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']



class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self, debug=False):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            if debug:
                print '-- HProxy:Get: {}'.format(name)
            self.instance = self.classtype(name, self.root_file, debug)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None,
                  debug=False):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
        if debug:
            print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get(debug) for his in histo_df['histo'].values]
        return histo, labels, text


In [3]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [4]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v76t'


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v75t', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v75t', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', 'v77', 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', 'v75', 'V9')
    ]


all_tpsets = {'DEF': 'dRC3d',
              'DEFCalib': 'NNDR Calib v1',
              'DEFNC': 'dRC3d + new Th',
              'HMvDR': 'HistoMaxC3d + dR(layer)',
              'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
              'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
              'EG': 'EG',
              'TkEG': 'TkEG',
              'TkEle': 'TkEle',
              'TkIsoEle': 'TkIsoEle',
              'L1Trk': 'L1Track'}


gen_part_selections: 10


Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v76t.root does not exist


In [5]:
# %load settings.py
# === samples =====================================================
samples = []

samples = samples_nugunrates_V8
samples += samples_nugunrates_V9


sample = 'V9'

do_rate = True 

# === TP ==========================================================
tps = ['EG', 'TkEG', 'TkEle', 'TkIsoEle']

# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['Em', 'all']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3', 'EGq2EtaBC', 'EGq3EtaBC']

tp_select['TkEG'] = get_label_dict(selections.tkeg_rate_selections).keys()
tp_select['TkEle'] = ['EGq2EtaBC', 'EGq3EtaBC']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


--- TPs: 
['EG', 'TkEG', 'TkEle', 'TkIsoEle']
--- TP selections:
{   'DEF': ['Em', 'all'],
    'DEFNC': ['Em', 'all'],
    'EG': ['EGq2', 'EGq3', 'EGq2EtaBC', 'EGq3EtaBC'],
    'HMvDR': ['Em', 'all'],
    'HMvDRNC0': ['Em', 'all'],
    'HMvDRNC1': ['Em', 'all'],
    'L1Trk': ['all', 'Pt2', 'Pt10'],
    'TkEG': [   'EGq2EtaBCM1P10',
                'EGq3EtaBCM1P2',
                'EGq2EtaCM1P10',
                'EGq3EtaBCM1P5',
                'EGq3EtaB',
                'EGq3EtaC',
                'EGq3EtaD',
                'EGq2EtaDM1P5S',
                'EGq2EtaBCM1P2S',
                'EGq2EtaCM1P2S',
                'EGq3EtaBM1P10S',
                'EGq3EtaBCM1',
                'EGq3EtaBM1P5S',
                'EGq2EtaC',
                'EGq2EtaBM1P2',
                'EGq2EtaBM1P5',
                'EGq2EtaD',
                'EGq3M1P10',
                'EGq3EtaBM1',
                'EGq2M1',
                'EGq2EtaBM1',
                'EGq3EtaCM1P2',
                'E

In [6]:
tpset_selections


{'EGq2': 'q2',
 'EGq2EtaB': 'q2, 1.52 < |#eta^{L1}| <= 1.7',
 'EGq2EtaBC': 'q2, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq2EtaBCD': 'q2, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq2EtaBCDM1': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07',
 'EGq2EtaBCDM1P10': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2EtaBCDM1P10S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2EtaBCDM1P2': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 2GeV',
 'EGq2EtaBCDM1P2S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 2GeV',
 'EGq2EtaBCDM1P5': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 5GeV',
 'EGq2EtaBCDM1P5S': 'q2, 1.52 < |#eta^{L1}| <= 2.8, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 5GeV',
 'EGq2EtaBCM1': 'q2, 1.52 < |#eta^{L1}| <= 2.4, |#Delta#phi| <0.08 & #DeltaR < 0.07',
 'EG

In [7]:
gen_selections

{'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [8]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, [])


hplot.cache_histo(classtype=histos.RateHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=None)



<drawingTools.Sample instance at 0x1153aec68>
<drawingTools.Sample instance at 0x1153aec68> EG EGq2EtaBCM1P10 None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaBCM1P2 None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3Iso0p3 None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3M1P5S None
<drawingTools.Sample instance at 0x1153aec68> EG EGq2EtaCM1P10 None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaBCM1P5 None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaBM1P2S None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaB None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaC None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaD None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaDM1P10S None
<drawingTools.Sample instance at 0x1153aec68> EG EGq3EtaCM1P5 None
<drawingTools.Sample instance at 0x1153aec68> EG EGq2EtaBCM1P5S None
<drawingTools.Sample instance at 0x1153aec68> EG EGq2EtaDM1P5S None
<drawingTools.Sample i

<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2EtaD None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2Iso0p2EtaBC None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2M1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2EtaBM1P2 None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2Iso0p2EtaD None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2EtaCM1P5 None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2EtaCM1P2 None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2Iso0p2EtaC None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq2EtaCM1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq3M1P10 None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq3EtaBM1P5 None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq3EtaBCDM1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq3EtaBCM1P2S None
<drawingTools.Sample instance at 0x1153aec68> TkEG EGq3EtaBM1 None
<drawingTools.Sample instance at 

<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2Iso0p2EtaBC None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2M1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2EtaBM1P2 None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2Iso0p2EtaD None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2EtaCM1P5 None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2EtaCM1P2 None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2Iso0p2EtaC None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2EtaCM1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq3M1P10 None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq3EtaBM1P5 None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq3EtaBCDM1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq3EtaBCM1P2S None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq3EtaBM1 None
<drawingTools.Sample instance at 0x1153aec68> TkEle EGq2EtaBCDM1P5 None
<drawingTools

<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2EtaBM1P2 None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2Iso0p2EtaD None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2EtaCM1P5 None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2EtaCM1P2 None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2Iso0p2EtaC None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2EtaCM1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq3M1P10 None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq3EtaBM1P5 None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq3EtaBCDM1P5S None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq3EtaBCM1P2S None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq3EtaBM1 None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2EtaBCDM1P5 None
<drawingTools.Sample instance at 0x1153aec68> TkIsoEle EGq2M1 None
<drawingTools.Sample instance at 0x1153aec68> TkIs

<drawingTools.Sample instance at 0x1153aed40> EG EGq3Iso0p3EtaB None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3Iso0p3EtaC None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3Iso0p3EtaD None
<drawingTools.Sample instance at 0x1153aed40> EG EGq2EtaDM1 None
<drawingTools.Sample instance at 0x1153aed40> EG EGq2Iso0p3EtaBCD None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3Iso0p3EtaBC None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3EtaCM1P2 None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3EtaBC None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3EtaDM1P2S None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3M1 None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3 None
<drawingTools.Sample instance at 0x1153aed40> EG EGq2 None
<drawingTools.Sample instance at 0x1153aed40> EG EGq2EtaBCD None
<drawingTools.Sample instance at 0x1153aed40> EG EGq3EtaBCDM1 None
<drawingTools.Sample instance at 0x1153aed40> EG EGq2EtaBCDM1P2S None
<drawin

<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaBCD None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq3EtaBCDM1 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaBCDM1P2S None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq3EtaDM1P5 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaCM1P10S None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq3EtaBM1P2 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaDM1P2S None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq3EtaCM1P10 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq3EtaDM1P10 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaBM1P5S None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaBCM1P5 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaDM1P2 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq3Iso0p2 None
<drawingTools.Sample instance at 0x1153aed40> TkEG EGq2EtaDM1P5 None
<drawingTools.Sample instanc

<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaBCD None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq3EtaBCDM1 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaBCDM1P2S None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq3EtaDM1P5 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaCM1P10S None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq3EtaBM1P2 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaDM1P2S None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq3EtaCM1P10 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq3EtaDM1P10 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaBM1P5S None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaBCM1P5 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaDM1P2 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq3Iso0p2 None
<drawingTools.Sample instance at 0x1153aed40> TkEle EGq2EtaDM1P5 None
<drawingTools.

<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq3EtaBM1P2 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq2EtaDM1P2S None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq3EtaCM1P10 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq3EtaDM1P10 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq2EtaBM1P5S None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq2EtaBCM1P5 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq2EtaDM1P2 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq3Iso0p2 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq2EtaDM1P5 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq2EtaBCM1P2 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq3EtaBCD None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq3EtaBCM1P10 None
<drawingTools.Sample instance at 0x1153aed40> TkIsoEle EGq3EtaBCDM1P5 None
<drawingTools.Sample instance at 0x1153aed40>

In [9]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V8,PU200,EG,EGq2EtaBCM1P10,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x114920ea8>
1,V8,PU200,EG,EGq3EtaBCM1P2,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x102cda710>
2,V8,PU200,EG,EGq3Iso0p3,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1041f6c20>
3,V8,PU200,EG,EGq3M1P5S,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11557f8c0>
4,V8,PU200,EG,EGq2EtaCM1P10,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x114920758>
5,V8,PU200,EG,EGq3EtaBCM1P5,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x10433f5a8>
6,V8,PU200,EG,EGq3EtaBM1P2S,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x11557fe18>
7,V8,PU200,EG,EGq3EtaB,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x114920d40>
8,V8,PU200,EG,EGq3EtaC,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x10433f758>
9,V8,PU200,EG,EGq3EtaD,None,python.l1THistos.RateHistos,<drawingTools.HProxy instance at 0x1155a9c20>


In [10]:
print dir(histos.RateHistos)

['__doc__', '__init__', '__module__', '__repr__', 'fill', 'fill_norm', 'normalize', 'write']


In [11]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        print 'Normalizing: TP: {}, sel: {}'.format(tp, tp_sel)
        hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)


Normalizing: TP: EG, sel: EGq2
normalize to 31000
Normalizing: TP: EG, sel: EGq3
normalize to 31000
Normalizing: TP: EG, sel: EGq2EtaBC
normalize to 31000
Normalizing: TP: EG, sel: EGq3EtaBC
normalize to 31000
Normalizing: TP: TkEG, sel: EGq2EtaBCM1P10
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaBCM1P2
normalize to 31000
Normalizing: TP: TkEG, sel: EGq2EtaCM1P10
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaBCM1P5
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaB
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaC
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaD
normalize to 31000
Normalizing: TP: TkEG, sel: EGq2EtaDM1P5S
normalize to 31000
Normalizing: TP: TkEG, sel: EGq2EtaBCM1P2S
normalize to 31000
Normalizing: TP: TkEG, sel: EGq2EtaCM1P2S
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaBM1P10S
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaBCM1
normalize to 31000
Normalizing: TP: TkEG, sel: EGq3EtaBM1P5S
normalize to 31000
Normalizing

#### 3D clusters matched to GEN

In [12]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_norm for his in hsets], labels, text=text)
         

In [13]:
for tp in ['EG', 'TkEle', 'TkIsoEle']:
    for tp_sel in tp_select[tp]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None, debug=False)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_pt for his in hsets], labels, text=text, logy=True)
         

normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000


In [ ]:
hsets_new = []
labels_new = []
# hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', 'TkEle', 'EGq2', None, debug=False)
# for hist in hsets:
#     hist.normalize(31000)
# hsets_new.extend(hsets)
# labels_new.append('TkEle')

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', 'TkEG', ['EGq2EtaBCM1P10', 'EGq2EtaBCM1P5', 'EGq2EtaBCM1P2'], None, debug=False)
for hist in hsets:
    hist.normalize(31000)

hsets_new.extend(hsets)
labels_new.extend(labels)

draw([his.h_pt for his in hsets_new], labels=['p_{T}^{trk} > 10GeV','p_{T}^{trk} > 5GeV','p_{T}^{trk} > 2GeV'], text='PU200, EM, 1.52<|#eta^{L1}|<2.4', logy=True)


In [61]:

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', ['TkEle', 'TkIsoEle'], ['EGq2Iso0p2EtaBC'], None, debug=False)
for hist in hsets:
    hist.normalize(31000)


draw([his.h_pt for his in hsets], labels=labels, text=text, min_x=0, max_x=40, min_ratio_y=0, max_ratio_y=2, logy=True, do_ratio=True)


<ROOT.TCanvas object ("c_1e08ba") at 0x7fccf7a5dbb0>

In [64]:
hsets_new = []
labels_new = []
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', 'EG', 'EGq2EtaBC', None, debug=True)
for hist in hsets:
    hist.normalize(31000)
hsets_new.extend(hsets)
labels_new.append('EG, q2')

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', 'TkEle', 'EGq2EtaBC', None, debug=True)
for hist in hsets:
    hist.normalize(31000)
hsets_new.extend(hsets)
labels_new.append('TkEle, q2')

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', 'TkEle', 'EGq2Iso0p2', None, debug=True)
for hist in hsets:
    hist.normalize(31000)
hsets_new.extend(hsets)
labels_new.append('TkEle, q2, Isolated')


hsets_new.extend(hsets)
labels_new.extend(labels)
print len(hsets_new)
draw([his.h_pt for his in hsets_new], labels=labels_new, text='PU200, EM, 1.52<|#eta^{L1}|<2.4', logy=True)


   sample     pu  tp     tp_sel gen_sel                    classtype  \
94     V8  PU200  EG  EGq2EtaBC    None  python.l1THistos.RateHistos   

                                            histo  
94  <drawingTools.HProxy instance at 0x1155cdf80>  
    sample     pu     tp     tp_sel gen_sel                    classtype  \
334     V8  PU200  TkEle  EGq2EtaBC    None  python.l1THistos.RateHistos   

                                             histo  
334  <drawingTools.HProxy instance at 0x1155f0128>  
    sample     pu     tp      tp_sel gen_sel                    classtype  \
357     V8  PU200  TkEle  EGq2Iso0p2    None  python.l1THistos.RateHistos   

                                             histo  
357  <drawingTools.HProxy instance at 0x1155f8ea8>  
4


<ROOT.TCanvas object ("c_a08af6") at 0x7fccf79e9d70>

In [ ]:
hsets_new = []
labels_new = []
# hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', 'TkEle', 'EGq2', None, debug=False)
# for hist in hsets:
#     hist.normalize(31000)
# hsets_new.extend(hsets)
# labels_new.append('TkEle')

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', 'TkEG', ['EGq2EtaBCM1P10', 'EGq2EtaBCM1P5S', 'EGq2EtaBCM1P2S'], None, debug=False)
for hist in hsets:
    hist.normalize(31000)

hsets_new.extend(hsets)
labels_new.extend(labels)

draw([his.h_pt for his in hsets_new], labels=['p_{T}^{trk} > 10GeV','p_{T}^{trk} > 5GeV & #stubs < 3','p_{T}^{trk} > 2GeV & #stubs < 3'], text='PU200, EM, 1.52<|#eta^{L1}|<2.4', logy=True)


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V8', 'V9'], 'PU200', ['EG'], ['EGq2EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)


In [15]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', ['EG'], ['EGq2EtaBC','EGq3EtaBC'], None, debug=True)
for hist in hsets:
    hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)


   sample     pu  tp     tp_sel gen_sel                    classtype  \
62     V8  PU200  EG  EGq3EtaBC    None  python.l1THistos.RateHistos   
94     V8  PU200  EG  EGq2EtaBC    None  python.l1THistos.RateHistos   

                                            histo  
62  <drawingTools.HProxy instance at 0x1155c1e18>  
94  <drawingTools.HProxy instance at 0x1155cdf80>  


<ROOT.TCanvas object ("c_11c607") at 0x7fccf7b234e0>

In [16]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'V8', 'PU200', ['EG'], ['EGq2EtaD','EGq3EtaD'], None, debug=True)
for hist in hsets:
    hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)


   sample     pu  tp    tp_sel gen_sel                    classtype  \
9      V8  PU200  EG  EGq3EtaD    None  python.l1THistos.RateHistos   
37     V8  PU200  EG  EGq2EtaD    None  python.l1THistos.RateHistos   

                                            histo  
9   <drawingTools.HProxy instance at 0x1155a9c20>  
37  <drawingTools.HProxy instance at 0x1155b7758>  
-- HProxy:Get: EG_EGq3EtaD
[(<ROOT.TH1F object ("EG_EGq3EtaD_norm") at 0x7fccf79a8990>, 'EG_EGq3EtaD_norm'), (<ROOT.TH1F object ("EG_EGq3EtaD_pt") at 0x7fccf79a8d80>, 'EG_EGq3EtaD_pt'), (<ROOT.TH2F object ("EG_EGq3EtaD_ptVabseta") at 0x7fccf3acde00>, 'EG_EGq3EtaD_ptVabseta')]
-- HProxy:Get: EG_EGq2EtaD
[(<ROOT.TH1F object ("EG_EGq2EtaD_norm") at 0x7fccf7a0ed40>, 'EG_EGq2EtaD_norm'), (<ROOT.TH1F object ("EG_EGq2EtaD_pt") at 0x7fccf7a0f3d0>, 'EG_EGq2EtaD_pt'), (<ROOT.TH2F object ("EG_EGq2EtaD_ptVabseta") at 0x7fccf2c89a00>, 'EG_EGq2EtaD_ptVabseta')]
normalize to 31000
normalize to 31000


<ROOT.TCanvas object ("c_2a4977") at 0x7fccf7a0faf0>

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEle'], ['EGq2EtaBC','EGq3EtaBC'], None, debug=True)
for hist in hsets:
    hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEle'], ['EGq2Iso0p2','EGq3Iso0p2'], None)
for hist in hsets:
    hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEG'], ['EGq2M1','EGq2M3', 'EGq2M1P', 'EGq2M1S'], None)
for hist in hsets:
    hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)
       

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEle'], ['EGq2'], None)
for hist in hsets:
    hist.normalize(31000)
hsets1, labels1, text1 = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEG'], ['EGq2M1P'], None)
for hist in hsets1:
    hist.normalize(31000)
draw([hsets[0].h_pt, hsets1[0].h_pt], ['TkEle', 'TkEG'], text=text, logy=True, do_ratio=True)


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEle'], ['EGq2Iso0p2'], None)
for hist in hsets:
    hist.normalize(31000)
hsets1, labels1, text1 = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkIsoEle'], ['EGq2Iso0p2'], None)
for hist in hsets1:
    hist.normalize(31000)
draw([hsets[0].h_pt, hsets1[0].h_pt], ['TkEle', 'TkIsoEle'], text=text, logy=True, do_ratio=True)


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEle'], ['EGq2Iso0p3'], None)
for hist in hsets:
    hist.normalize(31000)
hsets1, labels1, text1 = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkIsoEle'], ['EGq2Iso0p3'], None)
for hist in hsets1:
    hist.normalize(31000)
draw([hsets[0].h_pt, hsets1[0].h_pt], ['TkEle', 'TkIsoEle'], text=text, logy=True, do_ratio=True)


In [ ]:
for tp in ['TkEG']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, ['EGq2', 'EGq2M2','EGq2M2s','EGq2M3','EGq2M3s','EGq2M4',], None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_pt for his in hsets], labels, text=text, logy=True)
         

In [ ]:
for tp in ['TkEG']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, ['EGq3', 'EGq3M2','EGq3M2s','EGq3M3','EGq3M3s','EGq3M4',], None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_pt for his in hsets], labels, text=text, logy=True)


In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, ['EmEtaABC', 'EmEtaBC'], None)
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    hist_list = [his.h_pt for his in hsets]
    drawSame(hist_list, labels, text=text, logy=True)
    hRatio = hist_list[1].Clone(uuid.uuid4().hex[:6])
    stuff.append(hRatio)
    hRatio.Divide(hist_list[0])
    drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['DEF','DEFCalib'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
drawSame(hist_list, labels, text=text, logy=True)
hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
stuff.append(hRatio)
hRatio.Divide(hist_list[1])
drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

### Comparisono baseline vs Manos EGID

In [ ]:
for tp in ['DEF']:
    for etaregion in [['EmEtaBC', 'Emv1EtaBC'], ['EmEtaB', 'Emv1EtaB'], ['EmEtaC', 'Emv1EtaC'], ['EmEtaD', 'Emv1EtaD'], ['EmEtaE', 'Emv1EtaE']]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, etaregion, None)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        hist_list = [his.h_pt for his in hsets]
        drawSame(hist_list, labels, text=text, logy=True)
        hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
        stuff.append(hRatio)
        hRatio.Divide(hist_list[1])
        drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        # drawSame([his.h_ptVabseta for his in hsets], labels, text=text)
  

In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 11) for his in hsets], labels, text=text+', p_{T}^{L1}>10GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')



In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 21) for his in hsets], labels, text=text+', p_{T}^{L1}>20GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 31) for his in hsets], labels, text=text+', p_{T}^{L1}>30GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 41) for his in hsets], labels, text=text+', p_{T}^{L1}>40GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')
